In [1]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths
import pandas as pd
import importlib
from src.data import data_loader
import json

## diagnoses.csv

- contains the disease labels for a given rid.
- Question: there are sometimes multiple diagnoses per rid

In [2]:
def get_nested_csv(dir_name: str, file_name: str):
    """
    Returns a joint pandas dataframe from the files matching file_name
    in all the different import dates subdirectories of the directory
    specified by dir_name
    
    :param dir_name: The name of the directory (e.g. "seantis")
    :param file_name: The name of the csv file to be read. (e.g. "diagnoses.csv")
    """
    list_dfs = []

    for root, dirs, files in os.walk(os.path.join(paths.DATA_PATH_RAW, dir_name)):
        
        if root.split(os.sep)[-1].startswith("imported_20210507"):
            try:
                _df = pd.read_csv(os.path.join(root, file_name))
                list_dfs.append(_df)
            except FileNotFoundError:
                print(f"File not found in: {root}")
                continue
            except UnicodeDecodeError:
                print(f"UnicodeDecodeError in: {root}")
                continue

    df = pd.concat(list_dfs)
    return df

In [3]:
# Diagnoses.csv files
diagnoses = get_nested_csv("seantis", "diagnoses.csv")
display(diagnoses.head())

# Count rows
print("Number of rows: ", len(diagnoses))

# Count unique research_ids
print("Number of unique research_ids: ", diagnoses.research_id.nunique())

# rids that have 2 or more occurences
print("Number of rids that have 2 or more occurences: ", diagnoses[diagnoses.research_id.duplicated()].research_id.nunique())

# Occurences per rid
print("Occurences per rid: ", diagnoses.research_id.value_counts())

# Occurences per disease
print("Occurences per disease: ", diagnoses.disease.value_counts())

# Examples of duplicate research_ids
display(diagnoses[diagnoses.research_id.duplicated()].iloc[:2])

# List of rids
diagnoses_rids = diagnoses.research_id.unique()

,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
0,2011-10,confirmed,Progressive multifokale Leukencephalopathie (PML),2011-10,A0684D32-19C0-4538-AFDE-52959CCD8B63
1,2002-08,confirmed,relapsing_remitting_multiple_sclerosis,2002-06,A0684D32-19C0-4538-AFDE-52959CCD8B63
2,2011-11,confirmed,St.n. symptomatischer Epilepsie mit einfach fo...,2011-11,A0684D32-19C0-4538-AFDE-52959CCD8B63
3,2008-10,confirmed,relapsing_remitting_multiple_sclerosis,2003,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
4,2000,NaN,relapsing_remitting_multiple_sclerosis,2000,EF6AFC6C-BEB1-43B9-A5F4-79F321FF100C


Number of rows:  387
Number of unique research_ids:  333
Number of rids that have 2 or more occurences:  53
Occurences per rid:  research_id
A0684D32-19C0-4538-AFDE-52959CCD8B63    3
E504030A-CA83-412F-ADC4-C2596ED43B38    2
E3991607-3093-4168-B53F-A05DAA8A51EB    2
D10D6E44-36BD-4195-BB8A-210C363F8420    2
DFC51E69-4E29-467F-AF71-BBFE79F6778D    2
                                       ..
14F6D217-180C-4EF5-9964-488D06EB6693    1
09038A97-4134-438B-8FE6-3B6F88BE4968    1
810EEFD7-1C70-46E8-912B-0C836C45D93D    1
8532876E-4D7D-47F5-9BB6-4D20F0D8FA1B    1
75776E10-BE56-4697-887E-BFEBA373BB5B    1
Name: count, Length: 333, dtype: int64
Occurences per disease:  disease
relapsing_remitting_multiple_sclerosis                                              284
clinically_isolated_syndrome                                                         45
secondary_progressive_multiple_sclerosis                                             27
primary_progressive_multiple_sclerosis                       

,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
1,2002-08,confirmed,relapsing_remitting_multiple_sclerosis,2002-06,A0684D32-19C0-4538-AFDE-52959CCD8B63
2,2011-11,confirmed,St.n. symptomatischer Epilepsie mit einfach fo...,2011-11,A0684D32-19C0-4538-AFDE-52959CCD8B63


In [4]:
diagnoses[diagnoses.research_id == "2048FE8D-4DFF-4939-9739-1B5A470914DA"]

,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
309,1999,NaN,relapsing_remitting_multiple_sclerosis,1999,2048FE8D-4DFF-4939-9739-1B5A470914DA


### kisim_diagnoses.csv
- Nearly all rid have multiple diagnoses
- Some have up to 676 diagnoses. How do I match this to the diagnoses.csv file?

In [5]:
# Kisim Diagnoses 
kisim_diagnoses = get_nested_csv("seantis", "kisim_diagnoses.csv")
display(kisim_diagnoses.head())
print("Lenght of kisim_diagnoses: ", len(kisim_diagnoses))

#Count duplicate diagnosis ids
print("Duplicate diagnosis_id: ", kisim_diagnoses.diagnosis_id.duplicated().sum())

# Count unique research ids
print("Unique research_ids: ", kisim_diagnoses.research_id.nunique())

# Number of rid that have 2 or more occurences
print("Number of rids that have 2 or more occurences: : ", kisim_diagnoses[kisim_diagnoses.research_id.duplicated()].research_id.nunique())


,data_provider_code,diagnosis_code,diagnosis_code_system,diagnosis_code_system_version,diagnosis_date,diagnosis_encoded,diagnosis_id,diagnosis_label,diagnosis_ranking,diagnosis_role,LastUpdateDateTime,research_id
0,CHE-108.904.325,NaN,NaN,NaN,NaN,no,7024338|1,Kutane Karzinogenese \r\nSpinozelläre Karzinom...,primary,discharge,2021-01-19 08:34:58.0800000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
1,CHE-108.904.325,NaN,NaN,NaN,NaN,no,5255555|1,Aktuell kompensierte Harnblasensituation bei s...,primary,discharge,2016-10-07 14:06:05.2630000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
2,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6435406|1,Aktuell kompensierte Harnblasensituation bei s...,primary,discharge,2020-04-28 06:27:53.2300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
3,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6352273|5,"Schubförmige Multiple Sklerose\r\nES 03, ED 10...",secondary,discharge,2019-07-24 11:21:01.4300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
4,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6352273|4,Aktuell kompensierte Harnblasensituation bei s...,secondary,discharge,2019-07-24 11:21:01.4300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1


Lenght of kisim_diagnoses:  7238
Duplicate diagnosis_id:  0
Unique research_ids:  275
Number of rids that have 2 or more occurences: :  254


In [6]:
# Count number of diagnosis per research_id
print("Number of diagnosis per research_id: ")
display(kisim_diagnoses["research_id"].value_counts())

# Id "2048FE8D-4DFF-4939-9739-1B5A470914DA" has 676 diagnoses
display(kisim_diagnoses[(kisim_diagnoses.research_id == "2048FE8D-4DFF-4939-9739-1B5A470914DA")])

#Extract longest diagnosis_label from kisim_diagnoses[(kisim_diagnoses.research_id == "2048FE8D-4DFF-4939-9739-1B5A470914DA")]
print("Longest Diagnosis Length" ,kisim_diagnoses[(kisim_diagnoses.research_id == "2048FE8D-4DFF-4939-9739-1B5A470914DA")].diagnosis_label.str.len().max())

Number of diagnosis per research_id: 


research_id
2048FE8D-4DFF-4939-9739-1B5A470914DA    676
8BE4E431-291A-4722-B78F-3CD446550505    556
920241EB-EF90-40BC-9F5A-2D64CBA63FCC    308
61A25E27-F918-4545-BC27-5DEE14798C26    246
628E2D8D-84CC-40AB-8D3D-160725D214DB    159
                                       ... 
CB96C8EB-8D90-4D3A-B9D1-F02F1AD11377      1
5D3E2499-2EC9-4DAC-9146-A87913740D7A      1
A39CF12D-AB32-4F83-930F-43D17BD287AE      1
5D7FBBE1-CB5D-4741-AE7B-32A4C4CFA1A1      1
BF7EA6DC-B4E8-429B-9477-2E6B2BCFAD1F      1
Name: count, Length: 275, dtype: int64

,data_provider_code,diagnosis_code,diagnosis_code_system,diagnosis_code_system_version,diagnosis_date,diagnosis_encoded,diagnosis_id,diagnosis_label,diagnosis_ranking,diagnosis_role,LastUpdateDateTime,research_id
1792,CHE-108.904.325,NaN,NaN,NaN,NaN,no,4491852|3,Chronisches lumbospondylogenes Syndrom M54.4\r...,secondary,discharge,2015-02-17 12:02:24.4530000,2048FE8D-4DFF-4939-9739-1B5A470914DA
1793,CHE-108.904.325,NaN,NaN,NaN,NaN,no,4491852|2,Chronisches cervicobrachiales Syndrom linksbet...,secondary,discharge,2015-02-17 12:02:24.4530000,2048FE8D-4DFF-4939-9739-1B5A470914DA
1794,CHE-108.904.325,NaN,NaN,NaN,NaN,no,4491852|1,Lupoide Kollagenose mit/bei\r\n1.1 Kutaner Lup...,primary,discharge,2015-02-17 12:02:24.4530000,2048FE8D-4DFF-4939-9739-1B5A470914DA
1795,CHE-108.904.325,NaN,NaN,NaN,NaN,no,4503711|10,Nebendiagnosen der Haut\r\n··Aktinische Kerato...,secondary,discharge,2014-12-01 12:46:39.1430000,2048FE8D-4DFF-4939-9739-1B5A470914DA
1796,CHE-108.904.325,NaN,NaN,NaN,NaN,no,4503711|9,Chronischer schwerer Vitamin D-Mangel\r\n,secondary,discharge,2014-12-01 12:46:39.1430000,2048FE8D-4DFF-4939-9739-1B5A470914DA
...,...,...,...,...,...,...,...,...,...,...,...,...
6994,CHE-108.904.325,M54.4,ICD10,ICD-10-GM 2016,2018-12-17 16:40:00.0000000,yes,5265668|5|1,Lumboischialgie,secondary,discharge,2018-12-18 09:26:34.7730000,2048FE8D-4DFF-4939-9739-1B5A470914DA
6995,CHE-108.904.325,E55.9,ICD10,ICD-10-GM 2016,2018-12-17 16:40:00.0000000,yes,5265668|9|1,Vitamin-D-Mangel,secondary,discharge,2018-12-18 09:26:34.7730000,2048FE8D-4DFF-4939-9739-1B5A470914DA
6996,CHE-108.904.325,F32.1,ICD10,ICD-10-GM 2016,2018-12-17 16:40:00.0000000,yes,5265668|10|1,Mittelgradige depressive Episode,secondary,discharge,2018-12-18 09:26:34.7730000,2048FE8D-4DFF-4939-9739-1B5A470914DA
6997,CHE-108.904.325,M51.9,ICD10,NaN,2017-06-26 14:26:00.0000000,yes,6237818|3|1,"lumbo-vertebrales, -spondylogenes Syndrom",primary,discharge,2019-09-16 10:34:27.6200000,2048FE8D-4DFF-4939-9739-1B5A470914DA


Longest Diagnosis Length 5114


In [7]:
# Example for a kisim_diagnosis
kisim_diagnoses.diagnosis_label.iloc[0]

'Kutane Karzinogenese \r\nSpinozelläre Karzinome \r\n··Unterarm links distal/dorsal \r\n··Exzision durch Hausarzt vom 02.07.19: Verruköses pIattenepitheliales Carcinoma in situ der Haut mit herdförmiger Begleitentzündung sowie solarer Elastose der umgebenden Dermis. Läsion an einer seitlichen Resektionsfläche herdförmig Rand bildend. \r\n·Nachexzision durch Hausarzt \r\n'

In [8]:
# Overlap with diagnoses.csv rid
print("Overlap with diagnoses.csv rid: ", kisim_diagnoses.research_id.isin(diagnoses_rids).sum())

Overlap with diagnoses.csv rid:  6626


### reports_kisim_diagnoses

- How do these differ from seantis/kisim_diagnoses? Sometimes there are doubles?

In [9]:
# Reports diagnosis
reports_kisim_diagnoses = get_nested_csv("reports_with_struct_data",
                                         "reports_kisim_diagnoses.csv")
display(reports_kisim_diagnoses.head())

# Length of reports_kisim_diagnoses
print("Length of reports_kisim_diagnoses: ", len(reports_kisim_diagnoses))

# Count unique research_ids
print("Unique research_ids: ", len(reports_kisim_diagnoses.research_id.unique()))

# Number of rid that have 2 or more occurences
print("Number of rids that have 2 or more occurences: ", reports_kisim_diagnoses[reports_kisim_diagnoses.research_id.duplicated()].research_id.nunique())

# Count number of diagnosis per research_id
print("Number of diagnosis per research_id: ")
display(reports_kisim_diagnoses["research_id"].value_counts())

# Overlap with diagnoses.csv rid
print("Overlap with diagnoses.csv rid: ", reports_kisim_diagnoses.research_id.isin(diagnoses_rids).sum())

,data_provider_code,diagnosis_code,diagnosis_code_system,diagnosis_code_system_version,diagnosis_date,diagnosis_encoded,diagnosis_id,diagnosis_label,diagnosis_ranking,diagnosis_role,LastUpdateDateTime,research_id
0,CHE-108.904.325,NaN,NaN,NaN,NaN,no,5152802|1,Neurodegenerative Erkrankung unklarer Ätiologi...,primary,discharge,2016-07-12 08:33:38.6530000,5B60F0AC-2AB0-4F35-87A5-766ADD4812C6
1,CHE-108.904.325,NaN,NaN,NaN,NaN,no,5897600|2,Optikusneuropathie OS 01/2017 ED\r\n··rezidivi...,secondary,discharge,2018-04-11 09:06:31.5300000,8DB63228-EEFB-483D-A789-93F17047531D
2,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6146080|3,Diverse Allergien/Unverträglichkeiten u.a. Pen...,secondary,discharge,2019-03-04 13:50:26.8470000,60344C26-2777-40DD-AE67-945FE7EF84A7
3,CHE-108.904.325,NaN,NaN,NaN,NaN,no,3587988|11,Vitamin D-Mangel\r\n,secondary,discharge,2014-04-02 08:01:14.5230000,E88A076C-A03A-44EC-AB9C-6CD8286880E9
4,CHE-108.904.325,NaN,NaN,NaN,NaN,no,3632307|1,"Arterieller Hypertonus, ED 05/2012\r\n··Echoka...",primary,discharge,2014-04-02 08:10:17.8300000,209432DE-4590-4B24-8192-E0BB85871BF4


Length of reports_kisim_diagnoses:  87792
Unique research_ids:  1977
Number of rids that have 2 or more occurences:  1849
Number of diagnosis per research_id: 


research_id
914CDBAA-A5CB-4802-95D3-A48F71488A50    2068
F8B54AFD-9A93-4E26-A636-F3E926A5F165    1328
564D4B10-711B-4094-878E-A9F21CCCAC16    1191
A340F3A9-676F-4D1F-80F5-6C8F734D428E    1174
79C01C3B-7872-4C02-9B0B-0D7EDC8347B0    1155
                                        ... 
B42D9174-C0DF-49EA-B8FD-A722286C5BD7       1
CACC4AEC-E2E4-429E-8697-5FC53727E31A       1
7D1ADCD8-F468-4AEA-B1AD-85523A2F63EF       1
9CE987A3-68A5-42FF-B1D3-FB5FCEBE9CE6       1
C29DDABF-E6D6-4A07-ABE7-609571BAABD6       1
Name: count, Length: 1977, dtype: int64

Overlap with diagnoses.csv rid:  5455


In [10]:
# Checking if these are identical to kisim_diagnoses
pd.set_option("max_colwidth", None)
reports_kisim_diagnoses[reports_kisim_diagnoses.diagnosis_id == kisim_diagnoses.diagnosis_id.iloc[0]].diagnosis_label

20986    Kutane Karzinogenese \r\nSpinozelläre Karzinome \r\n··Unterarm links distal/dorsal \r\n··Exzision durch Hausarzt vom 02.07.19: Verruköses pIattenepitheliales Carcinoma in situ der Haut mit herdförmiger Begleitentzündung sowie solarer Elastose der umgebenden Dermis. Läsion an einer seitlichen Resektionsfläche herdförmig Rand bildend. \r\n·Nachexzision durch Hausarzt \r\n
Name: diagnosis_label, dtype: object

### Reports

- What is important here? Beurteilung? Zusatz? Form?

In [13]:
reports = pd.read_csv(os.path.join(paths.PROJECT_ROOT, r"preprocessed_nlp/midata-text-extraction/data/reports_with_struct_data/imported_20210507/all_info_reports.csv"))

/scratch/slurm-job.2936318/ipykernel_2625256/1351287029.py:1: DtypeWarning: Columns (16,17,19,71,72,73,75,76,77,78,79,80,81,82,83,84,86,87,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  reports = pd.read_csv(os.path.join(paths.PROJECT_ROOT, r"preprocessed_nlp/midata-text-extraction/data/reports_with_struct_data/imported_20210507/all_info_reports.csv"))


In [14]:
reports.head()

,Unnamed: 0,ARBF,AnArzt,Beilagen,BerichtsArt,BerichtsStatus,Beurteilung,Cl,ClBC,ClBCBeilagen,...,ZUSATZ_4,ZUSATZ_5,ZUSATZ_6,ZUSATZ_7,ZUSATZ_8,ZUSATZ_9,ZWAN,clDIAGS,report_filename,research_id
0,0,NaN,NaN,NaN,detaillierter Bericht,definitiv,"Die Patientin erscheint zur Besprechung und Festlegung der Basistherapie bei bekannter schubförmiger Multipler Sklerose. Anamnestisch kam es zu keinen schubverdächtigen Ereignissen seit der letzten Konsultation vom 22.07.2019. Nach ausführlicher Besprechung des Falles in unserem Neuroimmunoboard empfehlen wir der Patientin in erster Linie eine orale Basistherapie mit Tecfidera. Die Patientin wurde über potentielle Nebenwirkungen (gastrointestinale Probleme, insb. Diarrhoe, Sodbrennen, Übelkeit, vorrübergehende Errötung der Haut - Flush-, vorübergehender Transaminasenanstieg, Leukopenie mit erhöhter Infektneigung) informiert. Unter dieser Therapie müssen regelmässige Laborkontrollen erfolgen. Die Patientin möchte diese Kontrollen bei Ihnen durchführen lassen. Wir empfehlen, die Untersuchungen während der ersten drei Monate in monatlichem Rhythmus, danach alle drei Monate durchzuführen. \r\n\r\nWir werden die Patientin für ein ""Baseline""-MRI in 3-4 Monaten und für eine anschliessende Kontrolle und Befundbesprechung in unserer Sprechstunde aufbieten. Das weitere Prozedere erfolgt gemäss oben stehendem Schema. Für Rückfragen stehen wir Ihnen gerne zur Verfügung. \r\n",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Gehstrecke: weiterhin uneingeschränkt\r\nGehhilfe: keine\r\nPhysiotherapie: keine\r\nBlasenfunktion: gut\r\nMastdarmfunktion: gut\r\nVisus: gut\r\nFatigue: keine\r\nKonzentrationsstörungen: keine \r\n,Hauptdiagnosen,Report_MitrendS_F85C5D3E-26CD-459E-9E8A-CFA0ED3B48D7_2021.05.05-12-39-58.json,5B409EF5-12BE-4493-B0E4-A845146AC9C9
1,1,NaN,NaN,NaN,detaillierter Bericht,definitiv,"Im Rahmen der heutigen Telefonkonsultation konnten wir anamnestisch bei der Patientin einen stabilen Verlauf der MS ohne Hinweise auf neue schubartige Ereignisse eruieren. In der Bildgebung der Neuroachse zeigten sich zerebral keine neuen Läsionen und fraglich eine flaue KM-Aufnahme einer bestehenden Läsion auf Höhe HWK2/3, welche wir bei eingeschränkter Bildqualität und ohne Schubereignis am ehesten als Artefakt interpretieren. Die immunmodulatorische Therapie mit Tecfidera wurde soweit gut vertragen, es wurden keine vermehrten Atemwegsinfekte berichtet, laborchemisch (extern) zeigten sich normale Lymphozyten-, Leber-und Nierenwerte. Bei stabilem Verlauf der MS empfehlen wir somit eine Fortführung der Therapie. Im Falle einer unter Tecfidera eintretenden Schwangerschaft wird die Patientin sich telefonisch bei uns melden; entsprechend kann die Therapie nach Rücksprache mit uns dann pausiert werden. Zudem bitten wir um dreimonatliche hausärztliche Kontrollen von Blutbild (inkl. Diff-Blutbild), Leber- und Nierenwerten und Vitamin D. Insbesondere bei Abnahme der Leukozyten unter 3.0 G/l oder der Lymphozyten unter 0.5 G/l sowie bei deutlichem Kreatininanstieg bitten wir um Rücksprache. Es erfolgt eine Verlaufsbildgebung der Neuroachse und anschliessender klinischer Kontrolle in unserer neuroimmunologischen Sprechstunde in 6 Monaten. Für Rückfragen stehen wir jederzeit zur Verfügung.\r\n",NaN,NaN,NaN,...,NaN,NaN,"MRI Neuroachse 11.04.2020\r\n__Zur Befundung wurden die Wirbelsäulen MRT vom 04.07.2019 sowie die Schädel MRT-Voruntersuchung vom 02.07.2019 betrachtet. \r\nGehirn: \r\n· Vorbestehende, aktuell besser abgrenzbare fleckige Demyelinisierungen betont im Gyrus frontalis medius/superior rechts sowie im rechtsseitigen Aspekt des Forceps major und im periventrikulären Aspekt des Brachium pontis links nahe des Vermis (nur hier mit fraglicher diskreter Kontrastmittelaufnahme). \r\n· Ansonsten stationäre zahlreiche Demyelinisierungsherde. \r\n· Rückbildung der vormals zahlreichen Schrankenstörungen. Lediglich fragliche aktive Schrankenstörung auf Höhe der o.g. Läsion im Brachium pontis links, DD central vein sign. \r\n\r\n

In [15]:
reports.columns

Index(['Unnamed: 0', 'ARBF', 'AnArzt', 'Beilagen', 'BerichtsArt',
       'BerichtsStatus', 'Beurteilung', 'Cl', 'ClBC', 'ClBCBeilagen',
       'ClBCBeurteilung', 'ClBCDIAGS', 'ClBCEVDIAGS', 'ClBCFragestellung',
       'ClBCISTAT', 'ClBCMEDIEIN', 'ClBCMEDIHISTORY', 'ClBCNOTE', 'ClBCNSTAT',
       'ClBCNebenDIAGS', 'CreationDate', 'DAT1', 'DIAGS', 'EVDIAGS', 'FAMAN',
       'FORMNAME', 'Fahrtauglich', 'Form_EDSS', 'Fragestellung',
       'HeaderVorlage', 'ISTAT', 'JETZL', 'KonsArt', 'KonsDatum', 'KonsDatum2',
       'MEDIAUS', 'MEDIEIN', 'MEDIHISTORY', 'NACHKO', 'NOTE', 'NSTAT',
       'NebenDIAGS', 'PERSAN', 'PROC', 'PatAlter', 'Retoure', 'SCHLUSS',
       'SYSAN', 'Spezielles', 'Sprechstunde', 'TERMNR', 'Version10',
       'Version2', 'Version3', 'Version4', 'Version5', 'Version6', 'Version7',
       'Version8', 'Version9', 'Visum_AA', 'Visum_Chef', 'Visum_OA',
       'ZUSATZ_1', 'ZUSATZ_10', 'ZUSATZ_11', 'ZUSATZ_12', 'ZUSATZ_13',
       'ZUSATZ_14', 'ZUSATZ_15', 'ZUSATZ_16', 'ZUSATZ_1

In [18]:
with open(os.path.join(paths.PROJECT_ROOT, r"preprocessed_nlp/midata-text-extraction/data/reports_with_struct_data/imported_20210507/reports/Report_MitrendS_000B5446-F07C-4D9A-A336-39691B65AA7A_2021.05.05-12-39-58.json")) as f:
    file = json.load(f)

json.dumps(file, indent=4)

'{\n    "Header": {\n        "Hospital": "Spital",\n        "Clinic": "Neurologie",\n        "ShortAddress": "Untere Landstrasse",\n        "HeaderLayoutChange": "1"\n    },\n    "HeaderVorlage": "Klinik",\n    "Version2": "1",\n    "Version3": "1",\n    "Version4": "1",\n    "Version5": "1",\n    "Version6": "1",\n    "Version7": "1",\n    "CreationDate": "27.03.2017 14:01:42",\n    "PatAlter": "67",\n    "BerichtsStatus": null,\n    "DAT1": "Zurich, 27.03.2017/Stutz",\n    "Sprechstunde": "Sprechstunde f\\u00fcr Neuroimmunologie",\n    "KonsArt": "Behandlung",\n    "BerichtsArt": "detaillierter Bericht",\n    "KonsDatum": "27.03.2017",\n    "KonsDatum2": "27.03.2017",\n    "ClBCFragestellung": null,\n    "Fragestellung": "Zweitmeinung bez\\u00fcglich Therapieoptionen\\r\\n",\n    "clDIAGS": "Hauptdiagnosen",\n    "ClBCDIAGS": null,\n    "DIAGS": "Sekund\\u00e4r chronisch-progrediente Multiple Sklerose (Erstdiagnose der schubf\\u00f6rmigen MS 1998)\\r\\nAktuell: seit 6 Monaten eine Zu

In [ ]:
def flatten_json(d, parent_key='', sep='_'):
    """
    Flattens a nested JSON dict and handles the special case where there is an "Item" key with "@num" and "CONTENT" values.
    """
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            # Handle special case where there is an "Item" key with "@num" and "CONTENT" values
            if "Item" in v.keys():
                try:
                    item_num = v["Item"]["@num"]
                    item_content = v["Item"]["CONTENT"]
                    items.append((new_key + sep + "Item_" + item_num, item_content))
                    del v["Item"]
                except TypeError:
                    print(f"TypeError in {new_key}")
                    continue
                items.extend(flatten_json(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
        

def get_nested_json(path: str):
    with open(path) as f:
        data = json.load(f)
    return flatten_json(data)


def get_reports():
    """
    Returns the reports dataframe.
    """

    # research_id and filename are stored in reports_with_struct_data/reportfilename_researchid.csv files
    rid_filename = get_nested_csv("reports_with_struct_data", "reportfilename_researchid.csv")

    # Reports are in JSON format in reports_with_struct_data/reports
    # We need to 1) read the JSON files 2) flatten them to pd.DataFrame() format and 3) add the research_id from the csv files
    ls_reports = []

    for root, dirs, files in os.walk(os.path.join(paths.DATA_PATH_RAW, "reports_with_struct_data")):

        # All reports are in the reports folder
        if root.endswith("reports"):
            for file in files:
                try: 
                    # Get the research_id from the csv file
                    _rid = rid_filename[rid_filename.report_filename + ".json" == file].research_id.iloc[0]

                    # Read the JSON file
                    with open(os.path.join(root, file)) as f:
                        _json = json.load(f)
                    _df = pd.json_normalize(_json)

                    # Add the research_id
                    _df["research_id"] = _rid

                    # Append _df to list
                    ls_reports.append(_df)
                
                except IndexError:
                    print(f"No research_id found for {file}")
                    continue
    
    return pd.concat(ls_reports)

In [ ]:
df_reports = get_reports()
df_reports.columns

### medication.csv

In [19]:
# kisim_medication.csv
kisim_medication = get_nested_csv("seantis", "kisim_medication.csv")
display(kisim_medication.head())

# Length of kisim_medication
print("Length of kisim_medication: ", len(kisim_medication))

# Count unique research_ids
print("Unique research_ids: ", len(kisim_medication.research_id.unique()))

# Number of rid that have 2 or more occurences
print("Number of rids that have 2 or more occurences: ", kisim_medication[kisim_medication.research_id.duplicated()].research_id.nunique())

# Count number of medication per research_id
print("Number of medication entries per research_id: ")
display(kisim_medication["research_id"].value_counts())

# Overlap with diagnoses.csv rid
print("Overlap with diagnoses.csv rid: ", kisim_medication.research_id.isin(diagnoses_rids).sum())

,active_pharmaceutical_ingredient,data_provider_code,LastUpdateDateTime,medication_name,medication_prescription_date,medication_prescription_name,research_id
0,NaN,CHE-108.904.325,2020-03-07 09:57:27.7630000,"1 OP Medrol 32 mg \t1-0-0\n2 OP Keppra 500 mg\t3-0-3\n1 OP Urbanyl 10 mg \t0-0-0.5 noch für eine Woche\n1 OP Rivotril 0.5 mg\t1-1-0 noch für eine Woche, dann 1-1-1",2012-01-24 16:08:18.0000000,Rezept,A0684D32-19C0-4538-AFDE-52959CCD8B63
1,NaN,CHE-108.904.325,2020-03-07 09:58:13.9530000,1 OP Propanolol 20 mg 1-0-1\n2 OP Medrol 32 mg 1-0-0\n1 OP Calcimagon D3 forte 1-0-0,2012-02-07 14:22:55.0000000,Rezept,A0684D32-19C0-4538-AFDE-52959CCD8B63
2,NaN,CHE-108.904.325,2020-03-07 09:59:03.7300000,"1 OP Propranolol 40 mg 1-0-0.5\n2 OP Medrol 32 mg 1-0-0\n1 OP Calcimagon D3 forte 1-0-0\n1 OP (100 Stk) Keppra 500 mg 3-0-3\n1 OP (100 Stk.) Pantozol 40 mg\n1 OP Kalium Brause, 100 Stk.\n\n\n",2012-02-21 11:54:34.0000000,Rezept,A0684D32-19C0-4538-AFDE-52959CCD8B63
3,NaN,CHE-108.904.325,2020-03-07 09:59:28.7070000,1 OP Rivotril 0.5 mg\t1-1-1\n1 OP Propranolol 40 mg 1-0-0.5\n2 OP Medrol 16 mg 1-0-0,2012-02-29 10:23:45.0000000,Rezept,A0684D32-19C0-4538-AFDE-52959CCD8B63
4,NaN,CHE-108.904.325,2020-03-07 10:00:11.6070000,"Keppra 500 mg 3-0-2 max. 1 Woche, dann 2-0-2 für eine Woche",2012-03-09 15:11:03.0000000,Rezept,A0684D32-19C0-4538-AFDE-52959CCD8B63


Length of kisim_medication:  4435
Unique research_ids:  329
Number of rids that have 2 or more occurences:  297
Number of medication entries per research_id: 


research_id
2048FE8D-4DFF-4939-9739-1B5A470914DA    169
8BE4E431-291A-4722-B78F-3CD446550505    133
B8384A29-B60A-4573-B6EC-01BDFD299536     99
628E2D8D-84CC-40AB-8D3D-160725D214DB     77
C23380B6-FF91-46D5-88B6-E936F6DB07D9     76
                                       ... 
55AF64B4-D21A-43B1-B5CD-2305685BD634      1
0957B4DB-110F-4039-A101-C2790EE7BF53      1
99ECD55D-8577-4793-B365-23ECEE87BA21      1
888EC6C6-3566-4CB8-B54C-6F444FC98044      1
75776E10-BE56-4697-887E-BFEBA373BB5B      1
Name: count, Length: 329, dtype: int64

Overlap with diagnoses.csv rid:  4215


In [20]:
# EDSS Score
visits = get_nested_csv("seantis", "visits.csv")
display(visits.head())

# Count visits
print("Number of visits: ", len(visits))

# Count columns
print("Number of columns: ", len(visits.columns))

# Count unique research_ids
print("Unique research_ids: ", visits.research_id.nunique())

# Number of rid that have 2 or more occurences
print("Number of rids that have 2 or more occurences: ", visits[visits.research_id.duplicated()].research_id.nunique())

,alcohol_consumption,ambulation,amount_alcohol_consumption,babinski_sign_left_side,babinski_sign_right_side,bladder_bowel_sexual_dysfunction,bowel_bladder_functions,brainstem_diplopic_images,brainstem_face_paralysis,brainstem_functions,...,vegetativum_sexual_dysfunction,visit_date,visual_acuity,visual_functions,visual_functions_presence_temporal_pallor,visus_visual_disorder_colour_desaturation,visus_visual_disorder_laterality_uncertain,visus_visual_disorder_left,visus_visual_disorder_right,workflow_state
0,NaN,ambulation_0,NaN,NaN,NaN,NaN,bowel_bladder_functions_0,NaN,NaN,NaN,...,NaN,2020-02-14 00:00:00.0000000,NaN,NaN,visual_functions_0,NaN,NaN,NaN,NaN,closed
1,True,NaN,Am Wochenende,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2020-10-27 00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,open
2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2020-12-07 00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,True,True,open
3,NaN,ambulation_0,NaN,NaN,NaN,NaN,bowel_bladder_functions_1,NaN,NaN,NaN,...,NaN,2020-08-06 00:00:00.0000000,NaN,NaN,visual_functions_2,NaN,NaN,NaN,True,open
4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2020-08-06 00:00:00.0000000,NaN,NaN,NaN,NaN,NaN,NaN,True,open


Number of visits:  1639
Number of columns:  234
Unique research_ids:  333
Number of rids that have 2 or more occurences:  160


In [21]:
# visits.csv contains the EDSS score
display(visits.edss_score.head())

0    edss_0_0
1    edss_2_5
2    edss_2_0
3    edss_3_0
4    edss_4_0
Name: edss_score, dtype: object

### Exacerbations

In [22]:
exacerbations = get_nested_csv("seantis", "exacerbations.csv")
display(exacerbations.head())

# Count exacerbations
print("Number of exacerbations: ", len(exacerbations))

# Count columns

,exacerbation_end_date,exacerbation_start_date,research_id
0,2013-03,2013-02,A0684D32-19C0-4538-AFDE-52959CCD8B63
1,2011-08,2011-08,A0684D32-19C0-4538-AFDE-52959CCD8B63
2,2013-11,2013-08,A0684D32-19C0-4538-AFDE-52959CCD8B63
3,2011-08,2011-06,A0684D32-19C0-4538-AFDE-52959CCD8B63
4,2008,2003,A0684D32-19C0-4538-AFDE-52959CCD8B63


Number of exacerbations:  762


### MRI

In [23]:
# MRI
mri = get_nested_csv("seantis", "magnetic_resonance_images.csv")
display(mri.head())

# Count MRI
print("Number of MRI: ", len(mri))

# Count columns
print("Number of columns: ", len(mri.columns))

# Count unique research_ids
print("Unique research_ids: ", mri.research_id.nunique())

# Number of rid that have 2 or more occurences
print("Number of rids that have 2 or more occurences: ", mri[mri.research_id.duplicated()].research_id.nunique())

# Count number of MRI per research_id
print("Number of MRI per research_id: ")
display(mri["research_id"].value_counts())

# Overlap with diagnoses.csv rid
print("Overlap with diagnoses.csv rid: ", mri.research_id.isin(diagnoses_rids).sum())

,mri_contrast_medium,mri_date,mri_finding_place,mri_kind,mri_mcdonald_criteria,mri_normal,mri_number_new_enlarged_t2_lesions,mri_number_new_enlarged_t2_lesions_unit,mri_number_t1_gd_plus_lesions,mri_number_t1_gd_plus_lesions_unit,mri_number_t1_hypointense_lesions,mri_number_t1_hypointense_lesions_unit,mri_number_t2_lesions,mri_number_t2_lesions_unit,mri_pseudotumorous_lesions,mri_scanner,mri_type,mri_volume_t2_lesions,mri_volume_t2_lesions_unit,research_id
0,True,2020-08,NaN,mri_cranial,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mri_diagnostic,NaN,NaN,A107F5E3-CF59-4A4D-AF9A-77CFD47BDE83
1,True,2018-05-29,NaN,mri_cranial,NaN,NaN,0.0,U,0.0,U,NaN,NaN,NaN,NaN,False,NaN,mri_course,NaN,NaN,1C6C1C33-D734-4396-A515-34303C971AA6
2,True,2017-03-21,NaN,mri_spinal,NaN,NaN,0.0,U,0.0,U,NaN,NaN,0.0,U,False,NaN,mri_course,NaN,NaN,1C6C1C33-D734-4396-A515-34303C971AA6
3,True,2017-03-21,NaN,mri_cranial,NaN,NaN,0.0,U,0.0,U,NaN,NaN,NaN,NaN,False,NaN,mri_course,NaN,NaN,1C6C1C33-D734-4396-A515-34303C971AA6
4,True,2016-11-01,NaN,mri_cranial,NaN,NaN,0.0,U,0.0,U,NaN,NaN,NaN,NaN,False,NaN,mri_course,NaN,NaN,1C6C1C33-D734-4396-A515-34303C971AA6


Number of MRI:  1740
Number of columns:  20
Unique research_ids:  322
Number of rids that have 2 or more occurences:  198
Number of MRI per research_id: 


research_id
A0684D32-19C0-4538-AFDE-52959CCD8B63    38
189A2297-8248-470D-8378-66B37264856C    27
CF02C688-F65D-498F-95D9-FFDAEFC8AD54    26
86474070-A922-44CB-9829-372DDBE57AF2    23
E3991607-3093-4168-B53F-A05DAA8A51EB    22
                                        ..
610122E8-1889-4AB0-B325-FC09EA5DC783     1
55AF64B4-D21A-43B1-B5CD-2305685BD634     1
888EC6C6-3566-4CB8-B54C-6F444FC98044     1
4EED9C19-B74E-4550-A33B-7DCC8CC79C4F     1
A107F5E3-CF59-4A4D-AF9A-77CFD47BDE83     1
Name: count, Length: 322, dtype: int64

Overlap with diagnoses.csv rid:  1740


## My current understanding
- Taking the newest import (20210507) is sufficient
- I have three sources of input text (seantis/kisim_diagnoses.csv and reports_with_struct_data/reports_kisim_diagnoses.csv) and the reports?
- I have labels per text line in the preprocessed_nlp/labelling directory to classify text lines. But these only have the rid and not diagnosis id.

## Questions

### General
- Different imports, are all relevant?
- Are the dates listed in the files accuracte (could be used for matching?)
- There are a lot of data files, in different directories. Is the original data directory in: dataset/midatams/data? Because there is also some inside the preprocessed_nlp directory and then again in the midata extraction.
- What are the keywords you mention for the classifier 2?
- Is the seantis data done updating?
- The crf classifier uses probability matrices generated from random forrest. I have never seen this approach is there a paper for that? Or a documentation? Or can you explain shortly how you use the RF+CRF pipeline for both classifier 1&2? How does it help?

### diagnoses.csv
- What is the purpose of diagnoses.csv? Is this the labelling file for MS diagnosis?
- Is the diagnosis the "disease" column?
- Are you just interested in the 3 types of MS in the PDF (10 PPMS, 128 RRMS, 11 SPMS)
- Why are there duplicate research_ids? Are there multiple diagnoses per patient?
- What should we do if a patient has multiple diagnoses? Take last (from date)? 
- You used only 149 samples for training for the MS classifier?

### kisim_diagnoses.csv
- What is the purpose of kisim_diagnosis? Is this the text I should base the prediction on (in diagnosis_label)?
- What are the columns of interest here? diagnosis_label?
- Diagnosis ranking primary/secondary etc.? 
- What was labelled in diagnoses.csv? How do I match kisim_diagnosis.csv text to diagnosis.csv?
- What if there are multiple texts per rid, did you embed them, then aggregate and classify?

### reports_with_struct_data/reports_kisim_diagnoses
- Are these identical to the kisim_diagnoses? From the example it seems so. Or do they have overlap?
- What are the columns of interest here? diagnosis_label?

### Reports
- What exactly are the entries in the JSON that are of importance? ZUSATZ? FORM? Visum?
- How do I match the reports to diagnosis if there are multiple ids?

### seantis/medications.csv, seantis/kisim_medications.csv, rsd/kisim_medications.csv
- Are these the labels file for the medication task? Should I predict the medications based on seantis/kisim_diagnoses? or rsd/kisim_rsd_reports?
- How do I match the report to the medication label if I only have rids?
- Did you do prediction for this or just extracted the labels rule based? Do you want me to construct an approach that is not rule based and can be applied to seantis/medications.csv, seantis/kisim_medications.csv, rsd/kisim_medications.csv to extract medications? Or should I take these files as the labels and try to extract this info from seantis/kisim_diagnoses? or rsd/kisim_rsd_reports

### seantis/visits.csv
- Does this contain all the info about the edss score? Is this the labels file of the edss score?
- What if there are multiple entries per rid? How do I match it to the correct text?

### mri
- I can see the mri kind (spinal/cranial) but how do I check new/old and KM yes/no?

### Labelling
- There is a subdirectory preprocessed_nlp/labelling containing labelling for some text. What text exactly was labelled (from seantis/kisim_diagnoses)?
- There is only the rid to go off, but how would I match the labelling to patients with multiple diagnoses? How would I match the label to the correct text?

### Goal
- Do you want me to do predictions based on one text file, or aggregated text files (if multiple per rid) and then give one prediction for MS diagnosis, MRI, medication and edss score?

## Tasks:
- MS diagnosis: use diagnoses.csv for labels per rid, then predict from seantis/kisim_diagnoses.csv (diagnosis_label) or reports_with_struct_data/reports_kisim_diagnoses.csv (diagnosis_label) or the reports
- MRI: use magnetic_resonance_images.csv for labels, then predict from seantis/kisim_diagnoses.csv (diagnosis_label) or reports_with_struct_data/reports_kisim_diagnoses.csv (diagnosis_label) or the reports
- Medication: labels in seantis/medications.csv, seantis/kisim_medications.csv, rsd/kisim_medications.csv (need to be extracted rule based) then predict from seantis/kisim_diagnoses.csv (diagnosis_label) or reports_with_struct_data/reports_kisim_diagnoses.csv (diagnosis_label) or the report
- EDSS Score: labels in seantis/visits (edss_score) seantis/kisim_diagnoses.csv and reports (extract rule based) then predict from seantis/kisim_diagnoses.csv (diagnosis_label) or reports_with_struct_data/reports_kisim_diagnoses.csv (diagnosis_label) or the reports